# MAp side join

Map-side join makes sense when one of the tables — so called a fact table — is huge and the other table — so called a dimension table — is small enough to be broadcasted in memory.

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("spark-map-join").master("local[*]").getOrCreate()

To achived the Joined table result, we need to do the following:

# Step-1: Create Cache for Airports

Create a broadcast varaible for airports: first we create an RDD and then save it a dictionary[(key, value)] where key is an airport code, and value is associated values.

In [2]:
airports_data = [
    ("DTW", "Detroit Airport", "Detroit", "MI"),
    ("ORD", "Chicago O'Hare", "Chicago",  "IL"),
    ("JFK", "John F. Kennedy Int. Airport", "New York", "NY"),
    ("LAX", "Los Angeles Int. Airport", "Los Angeles", "CA"),
    ("SEA", "Seattle-Tacoma Int. Airport", "Seattle", "WA"),
    ("SFO", "San Francisco Int. Airport", "San Francisco", "CA")
]

In [3]:
airports_rdd = spark.sparkContext.parallelize(airports_data)\
.map(lambda tuple4: (tuple4[0], (tuple4[1],tuple4[2],tuple4[3])))

In [4]:
airports_dict = airports_rdd.collectAsMap()

In [6]:
airports_cache = spark.sparkContext.broadcast(airports_dict)
airports_cache.value

{'DTW': ('Detroit Airport', 'Detroit', 'MI'),
 'ORD': ("Chicago O'Hare", 'Chicago', 'IL'),
 'JFK': ('John F. Kennedy Int. Airport', 'New York', 'NY'),
 'LAX': ('Los Angeles Int. Airport', 'Los Angeles', 'CA'),
 'SEA': ('Seattle-Tacoma Int. Airport', 'Seattle', 'WA'),
 'SFO': ('San Francisco Int. Airport', 'San Francisco', 'CA')}

# Step-2: Create Cache for Airlines

Create a broadcast varaible for airlines: first we create an RDD and then save it a dictionary[(key, value)] where key is an airline code, and value is the name of airline.

In [10]:
airlines_data = [
    ("SW", "Southwest Airlines"),
    ("AA", "American Airlines"),
    ("DL", "Delta Airlines"),
    ("VX", "Virgin America")
]

In [11]:
airlines_rdd = spark.sparkContext.parallelize(airlines_data)\
.map(lambda tuple2: (tuple2[0], tuple2[1]))

In [12]:
airlines_dict = airlines_rdd.collectAsMap()

In [14]:
airlines_cache = spark.sparkContext.broadcast(airlines_dict)
airlines_cache
airlines_cache.value

{'SW': 'Southwest Airlines',
 'AA': 'American Airlines',
 'DL': 'Delta Airlines',
 'VX': 'Virgin America'}

# Step-3: Create Facts Table — Flights

Map each reacord of the Flights table and perform a simple join by lookup dictionaries created in Steps 1 & 2.

In [15]:
flights_data = [
    ("DTW", "ORD", "SW", "225",  "17:10"),
    ("DTW", "JFK", "SW", "355",  "8:20"),
    ("SEA", "JFK", "DL", "418",  "7:00"),
    ("SFO", "LAX", "AA", "1250", "7:05"),
    ("SFO", "JFK", "VX", "12",   "7:05"),
    ("JFK", "LAX", "DL", "424",  "7:10"),
    ("LAX", "SEA", "DL", "5737", "7:10")
]

In [16]:
flight_colums = ["from", "to", "airline", "flight_number", "departure"]

In [17]:
flights = spark.createDataFrame(flights_data, flight_colums)

In [18]:
flights.show(truncate=False)

+----+---+-------+-------------+---------+
|from|to |airline|flight_number|departure|
+----+---+-------+-------------+---------+
|DTW |ORD|SW     |225          |17:10    |
|DTW |JFK|SW     |355          |8:20     |
|SEA |JFK|DL     |418          |7:00     |
|SFO |LAX|AA     |1250         |7:05     |
|SFO |JFK|VX     |12           |7:05     |
|JFK |LAX|DL     |424          |7:10     |
|LAX |SEA|DL     |5737         |7:10     |
+----+---+-------+-------------+---------+



# Step-4: Map-Side Join:
Map each reacord of the Flights table — known as a facts table — and perform a simple join by lookup cache dictionaries created in Steps 1 & 2.

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [20]:
def get_airport(code):
    return airports_cache.value[code][1]

In [21]:
def get_airline(code):
    return airlines_cache.value[code]

In [24]:
airport_udf = udf(get_airport, StringType())

In [25]:
airline_udf = udf(get_airline, StringType())

In [26]:
flights.select(
        airport_udf("from").alias("from_city"),
        airport_udf("to").alias("to_city"),
        airline_udf("airline").alias("airline_name"),
        "flight_number", "departure").show(truncate=False)

+-------------+-----------+------------------+-------------+---------+
|from_city    |to_city    |airline_name      |flight_number|departure|
+-------------+-----------+------------------+-------------+---------+
|Detroit      |Chicago    |Southwest Airlines|225          |17:10    |
|Detroit      |New York   |Southwest Airlines|355          |8:20     |
|Seattle      |New York   |Delta Airlines    |418          |7:00     |
|San Francisco|Los Angeles|American Airlines |1250         |7:05     |
|San Francisco|New York   |Virgin America    |12           |7:05     |
|New York     |Los Angeles|Delta Airlines    |424          |7:10     |
|Los Angeles  |Seattle    |Delta Airlines    |5737         |7:10     |
+-------------+-----------+------------------+-------------+---------+



# Efficient Joins using Bloom filters